# Mix the tunes

In [1]:
import numpy as np 
import pandas as pd
import ffmpeg
from pathlib import Path

import glob
import os
import shutil

import IPython.display as ipd
from scipy.io import wavfile
from scipy.signal import resample

from pydub import AudioSegment
from pydub.effects import speedup

In [5]:
import librosa

### Pull all the available wavs that you can work with

In [2]:
def pull_available_wavs(source_dir, target_dir, extension):
    source_dir = os.path.expanduser(source_dir)
    target_dir = os.path.expanduser(target_dir)
    
    # Create target directory if it doesn't exist, make pattern to find wavs via glob match
    os.makedirs(target_dir, exist_ok=True)
    pattern = os.path.join(source_dir, f"*.{extension}")
    matching_files = glob.glob(pattern, recursive=False)

    # Add each file to a list
    available_wavs = [f for f in glob.glob(pattern) if os.path.isfile(f)]
    
    # Copy each file to target directory
    for file_path in matching_files:
        if os.path.isfile(file_path):
            filename = os.path.basename(file_path)
            target_path = os.path.join(target_dir, filename)
            shutil.copy2(file_path, target_path)

    return available_wavs

avail_wavs = pull_available_wavs("~/ohw25_proj_RiptideRemix/data/template_sounds", "~/ohw25_proj_RiptideRemix/contributor_folders/isabelle/audio_processing_mixers/available_wavs", "wav")

In [3]:
print("Playing ", avail_wavs[5])
ipd.Audio(avail_wavs[5])

Playing  /home/jovyan/ohw25_proj_RiptideRemix/data/template_sounds/bowhead_template.wav


### Loop file as many times as user requests, but will cut off at 60 seconds

In [4]:
def loop(input_file, output_file, num_loops, max_duration=60):
    sr, y = wavfile.read(input_file)

    num_samples = y.shape[0]
    max_samples = int(max_duration * sr)

    if y.ndim == 1:
        y_looped = np.tile(y, num_loops)
    else:
        y_looped = np.tile(y, (num_loops, 1))

    y_final = y_looped[:max_samples]

    wavfile.write(output_file, sr, y_final.astype(y.dtype))

    return 

loop(avail_wavs[5], "loop_exp.wav", 3)

In [5]:
ipd.Audio("loop_exp.wav")

### Increase speed / pitch

In [6]:
def pitch(input_file, output_file, factor):

    sr, y = wavfile.read(input_file)
    
    num_samples = int(len(y) / factor)
    y_resampled = resample(y, num_samples)
    
    wavfile.write(output_file, sr, y_resampled.astype(y.dtype))
    return
    

def speed(input_file, output_file, factor):
    audio = AudioSegment.from_wav(input_file)
    slower_audio = audio._spawn(audio.raw_data, overrides={"frame_rate": int(audio.frame_rate * factor)})
    slower_audio = slower_audio.set_frame_rate(audio.frame_rate)
    slower_audio.export(output_file, format="wav")
    
    return slower_audio

speed(avail_wavs[5], "speed_exp.wav", 2)

In [7]:
ipd.Audio("pitch_exp.wav")

### Increase amplitude

In [9]:
def amplitude(input_file, output_file, gain_factor):
    sr, y = wavfile.read(input_file)
    print(y[:10])
    
    y_amplified = y + gain_factor
    #print(y_amplified[:10])
    
    wavfile.write(output_file, sr, y)
    return

amplitude(avail_wavs[5], "amp_exp.wav", 1)

[-80  48  64  96 112  80  64 144  80  64]


In [10]:
ipd.Audio("amp_exp.wav")

In [4]:
#fix heightened background noise

In [3]:
import torchaudio
import torch
import torchaudio.functional as F
from torchaudio.transforms import PitchShift

In [4]:
waveform, sr = torchaudio.load(avail_wavs[5])
waveform = waveform.cpu()

transform = PitchShift(sample_rate = sr,
           n_steps = 4)

transform(waveform)

tensor([[-1.6903e-03,  1.6930e-03,  2.3923e-03,  ...,  9.9604e-05,
          7.1993e-04,  2.7387e-03]], grad_fn=<ViewBackward0>)

In [5]:
import torch
import torchaudio
print(torch.__version__)
print(torch.version.cuda)
print(torchaudio.__version__)

2.7.1
None
2.8.0


In [7]:
! pip uninstall torch torchaudio -y

Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: torchaudio 2.8.0
Uninstalling torchaudio-2.8.0:
  Successfully uninstalled torchaudio-2.8.0


In [9]:
! pip install torch==2.7.1 torchaudio==2.7.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 10.7 MB/s  0:00:26m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 17.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 40.3 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 42.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.2 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 7.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 14.5 MB/s  0:00:19m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 60.4 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 63.7 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 53.2 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 79.4

In [10]:
import torch
import torchaudio
print(torch.__version__)
print(torch.version.cuda)
print(torchaudio.__version__)

2.7.1
None
2.8.0
